In [ ]:
import pandas as pd
!pip install stanza
import stanza

In [ ]:
from bs4 import BeautifulSoup, Tag

In [ ]:
import json
import requests

In [ ]:
placez= pd.read_csv('/content/More_Places - Sheet1.csv') #this CSV file is all you need to run all of the code in this notebook
placez #manually created spreadsheet of placenames from map I used

In [ ]:
def get_pleiades_searchresults(places):
  for place in places:
    url=f'https://pleiades.stoa.org/search?SearchableText={place}&submit=Search' #searches the general search box in Pleiades
    r = requests.get(url)
    return (r.content)  #returns the html page of the search results


In [ ]:
placez_ = list(placez['Original Name'])
len(placez_)

173

In [ ]:
html_results= {}
for x in placez_:
  try:
    get_pleiades_searchresults(x)
    if get_pleiades_searchresults(x) in html_results:
      continue
    else:
      html_results[x]= get_pleiades_searchresults(x)  #have to double check the syntax but
  except:                                           #an error will be thrown if there is no search result which will stop the code
    pass                                            #this will have it skip any name with no results without breaking the loop

In [ ]:
def get_URI_list(soup):
  URIs= []
  for d in soup.find_all('dt'): #traversing down the html tag tree
    for b in d.find_all('span'):
      for h in b.find_all('a'):
        if h['href'] in URIs: #making sure we don't miss any URIs
          continue
        else:
          URIs.append(h['href'])
  return URIs



In [ ]:
uri_dict= {}
for x,y in html_results.items(): #making a dictionary
  soup= BeautifulSoup(y)        #ancient place name is the key, value is a list of every URI returned when we searched this name
  uri_dict[x]= get_URI_list(soup) #in Pleiades





In [ ]:
urlsDF= pd.DataFrame.from_dict(uri_dict,orient='index') #can convert dict into a dataframe
clipped_rez= urlsDF.loc[:, 0:4]
clipped_rez #there were a lot of search results #so I am cutting it off at 5
            #if I had more time, a better way to do this would be to manually check some of them and see which are more accurate

In [ ]:
clipped_rez.reset_index(inplace=True)

In [ ]:
urs= clipped_rez.rename(columns= {'index':'Ancient_Name', 0:'URI1',1:'URI2',2:'URI3',3:'URI4',4:'URI5'})
urs   #renaming columns

In [ ]:
urs.to_csv('/content/drive/MyDrive/GIS/FinalProject/ancientplacesURIs.csv') #saving just in case

In [ ]:
urs= pd.read_csv('/content/drive/MyDrive/GIS/FinalProject/ancientplacesURIs.csv')
urs

In [ ]:
def extract_pleiadesID(url):
    st= str(url)
    uri= st[33:] #using string slicing to remove all of the url except the ID, which is at the end
    return uri   #because they use consistent URIs, this slice works on every one so yay!





In [ ]:
urs['Pleiades_ID1'] = urs['URI1'].apply(lambda x: extract_pleiadesID(x)) #basically just applies above function to every value in the column
urs['Pleiades_ID2'] = urs['URI2'].apply(lambda x: extract_pleiadesID(x))
urs['Pleiades_ID3'] = urs['URI3'].apply(lambda x: extract_pleiadesID(x))
urs['Pleiades_ID4'] = urs['URI4'].apply(lambda x: extract_pleiadesID(x))
urs['Pleiades_ID5'] = urs['URI5'].apply(lambda x: extract_pleiadesID(x))
#extracting ID from URL
#there is probably a more efficient way to do this but

In [ ]:
#sanity check to make sure that code worked
urs

In [ ]:
import requests
import json

def get_coordinates1(pid):
    try:
        url = f'http://api.pleiades.stoa.org/places/{pid}/json'
        r = requests.get(url)
        cont = json.loads(r.content)
        coordinates = cont['features'][0]['geometry']['coordinates']
        if len(coordinates) == 2:
          return coordinates
        elif len(coordinates[0]) == 2:
          return coordinates[0]
        elif len(coordinates[0][0]) == 2:
          return coordinates[0][0]
        else:
            return coordinates[0][0][0]   #I could have used list comprehension but this worked and I was grumpy
    except:                               #some of the coordinates were lists and some were lists of lists
      pass



In [ ]:
urs['Coordinates1'] = urs['Pleiades_ID1'].apply(lambda x: get_coordinates1(x)) #applying function to values
urs['Coordinates2'] = urs['Pleiades_ID2'].apply(lambda x: get_coordinates1(x))
urs['Coordinates3'] = urs['Pleiades_ID3'].apply(lambda x: get_coordinates1(x))
urs['Coordinates4'] = urs['Pleiades_ID4'].apply(lambda x: get_coordinates1(x))
urs['Coordinates5'] = urs['Pleiades_ID5'].apply(lambda x: get_coordinates1(x))

In [ ]:
#sanity check
urs

In [ ]:
def get_feature_description(pid):
  try:
    url= f'http://api.pleiades.stoa.org/places/{pid}/json'
    r = requests.get(url)
    cont = json.loads(r.content)
    feat_desc= cont['features'][0]['properties']['description']  #tells us a bit more about the feature and data provenance
    return feat_desc
  except:
    pass


In [ ]:
urs['feature_description1']= urs['Pleiades_ID1'].apply(lambda x: get_feature_description(x))
urs['feature_description2']= urs['Pleiades_ID2'].apply(lambda x: get_feature_description(x))
urs['feature_description3']= urs['Pleiades_ID3'].apply(lambda x: get_feature_description(x))
urs['feature_description4']= urs['Pleiades_ID4'].apply(lambda x: get_feature_description(x))
urs['feature_description5']= urs['Pleiades_ID5'].apply(lambda x: get_feature_description(x))


In [ ]:
urs

In [ ]:
def get_place_description(pid):
  try:
    url= f'http://api.pleiades.stoa.org/places/{pid}/json'
    r = requests.get(url)
    cont = json.loads(r.content)
    p_desc= cont['description']
    return p_desc
  except:
    pass


In [ ]:
urs['place_description1']= urs['Pleiades_ID1'].apply(lambda x: get_place_description(x))
urs['place_description2']= urs['Pleiades_ID2'].apply(lambda x: get_place_description(x))
urs['place_description3']= urs['Pleiades_ID3'].apply(lambda x: get_place_description(x))
urs['place_description4']= urs['Pleiades_ID4'].apply(lambda x: get_place_description(x)) #this sometimes has actual interesting details, otherwise just
urs['place_description5']= urs['Pleiades_ID5'].apply(lambda x: get_place_description(x)) #has where it was cited (which is also good!)

In [ ]:
urs

In [ ]:
urs.to_csv('/content/drive/MyDrive/GIS/FinalProject/pleiadesDataC.csv') #this is much more data than we need for map making
                                                                        #but saving for the future

In [ ]:
coordinates1= pd.concat([urs['Ancient_Name'], urs['Pleiades_ID1'],urs['Coordinates1'],urs['place_description1']], axis=1)
coordinates2= pd.concat([urs['Ancient_Name'], urs['Pleiades_ID2'],urs['Coordinates2'],urs['place_description2']], axis=1)   #isolating coordinates from
coordinates3= pd.concat([urs['Ancient_Name'], urs['Pleiades_ID3'],urs['Coordinates3'],urs['place_description3']], axis=1)   #our much larger dataframe
coordinates4= pd.concat([urs['Ancient_Name'], urs['Pleiades_ID4'],urs['Coordinates4'],urs['place_description4']], axis=1)
coordinates5= pd.concat([urs['Ancient_Name'], urs['Pleiades_ID5'],urs['Coordinates5'],urs['place_description5']], axis=1)

In [ ]:
def return_latitude(lst):
  coordinates= lst[0]
  return coordinates

In [ ]:
def return_longitude(lst1):
  longitude= lst1[1]
  return longitude

In [ ]:
coordinates1['latitude1'] = coordinates1['Coordinates1'].dropna().apply(lambda x: return_latitude(x))
coordinates2['latitude2'] = coordinates2['Coordinates2'].dropna().apply(lambda x: return_latitude(x))  #once again, just applying function to get latitude to the dataframe
coordinates3['latitude3'] = coordinates3['Coordinates3'].dropna().apply(lambda x: return_latitude(x))
coordinates4['latitude4'] = coordinates4['Coordinates4'].dropna().apply(lambda x: return_latitude(x))
coordinates5['latitude5'] = coordinates5['Coordinates5'].dropna().apply(lambda x: return_latitude(x))

In [ ]:
coordinates1['longitude1'] = coordinates1['Coordinates1'].dropna().apply(lambda x: return_longitude(x))
coordinates2['longitude2'] = coordinates2['Coordinates2'].dropna().apply(lambda x: return_longitude(x))
coordinates3['longitude3'] = coordinates3['Coordinates3'].dropna().apply(lambda x: return_longitude(x)) #applying longitude function to dataframe
coordinates4['longitude4'] = coordinates4['Coordinates4'].dropna().apply(lambda x: return_longitude(x))
coordinates5['longitude5'] = coordinates5['Coordinates5'].dropna().apply(lambda x: return_longitude(x))

In [ ]:
coordinates1.to_csv('/content/drive/MyDrive/GIS/FinalProject/Coordinates_Data/Coordinates1.csv')
coordinates2.to_csv('/content/drive/MyDrive/GIS/FinalProject/Coordinates_Data/Coordinates2.csv')
coordinates3.to_csv('/content/drive/MyDrive/GIS/FinalProject/Coordinates_Data/Coordinates3.csv')
coordinates4.to_csv('/content/drive/MyDrive/GIS/FinalProject/Coordinates_Data/Coordinates4.csv')
coordinates5.to_csv('/content/drive/MyDrive/GIS/FinalProject/Coordinates_Data/Coordinates5.csv')

In [ ]:
def get_place_name(pid):  #I didn't end up using this function because it wasn't always super useful
  try:                    #But keeping it just in case it could be useful in the future
    url= f'http://api.pleiades.stoa.org/places/{pid}/json'
    r = requests.get(url)
    cont = json.loads(r.content)
    id_name= cont['features'][0]['id']
    return id_name
  except:
    pass


In [ ]:
urs['place_text']= urs['Pleiades_ID'].apply(lambda x: get_place_name(x))